#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 11, 5)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,23,B,SYNEX,"3,000",15.20,2,5pct,SET100,2022-02-02


In [3]:
orders_log = pd.read_csv('../data/orders-log.csv')
orders_log['trans'] = orders_log.apply(lambda row: categorise(row), axis=1)
orders_log['amount'] = orders_log.qty * orders_log.target
orders_log[orders_log.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount


In [4]:
orders_log.head().sort_values('spd',ascending=False).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,BANPU,-3,XXX,SET50,"7,500",12.00,12.30,-0.20,-1.60%,0,2022-09-14,Buy,"90,000.00"
4,B,CPNREIT,-4,HD,SET,"10,000",18.20,18.60,0.10,+0.54%,0,2022-08-17,Buy,"182,000.00"
3,B,BLA,-6,1L,SET50,"4,000",31.00,32.50,0.25,+0.78%,0,2022-08-23,Buy,"124,000.00"
0,B,BAM,-7,XXX,SET100,"6,000",14.50,15.20,-0.10,-0.65%,0,2022-04-29,Buy,"87,000.00"
2,B,BCH,-9,C21.46,SET100,"3,000",17.50,18.40,0.00,0.00%,0,2022-08-31,Buy,"52,500.00"


### End of Tables in the process

### Print to verify before upload file

In [5]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade     n...)

In [6]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

In [10]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)

df_orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,xdate
0,B,BCH,"3,000",17.50,"52,500.00",5pct,SET100,2022-08-31
1,B,KCE,"1,000",41.00,"41,000.00",5pct,SET50,2022-02-02
2,B,PTTGC,"3,000",41.25,"123,750.00",5pct,SET50,2022-09-02
3,B,RCL,"3,000",27.00,"81,000.00",5pct,SET100,2022-08-18
4,B,STA,"2,500",18.10,"45,250.00",5pct,SET100,2022-08-23
5,B,SYNEX,"3,000",15.20,"45,600.00",5pct,SET100,2022-02-02
6,S,MCS,"5,000",9.70,"48,500.00",5pct,SET,2022-09-03
7,S,SCC,300,340.00,"102,000.00",C398,SET50,2022-08-10


In [12]:
file_name = 'df_orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders.to_csv(output_file, header=True, index=False)
df_orders.to_csv(data_file, header=True, index=False)
df_orders.to_csv(box_file, header=True, index=False)
df_orders.to_csv(one_file, header=True, index=False)

### End of transactional process

In [8]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,MCS,"5,000",11.40,"57,000.00",52WL,SET,2,2022-03-15
1,S,GLOBAL,"4,500",18.60,"83,700.00",C18.40,SET50,2,2022-02-02
2,S,JASIF,"10,000",9.85,"98,500.00",C9.80,SET,2,2022-05-20


In [9]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B     57000.0
S    182200.0
Name: amount, dtype: float64

In [10]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,MCS,"5,000",11.40,"57,000.00",52WL,SET,2,2022-03-15


In [11]:
cash = 48_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(152000, 57000.0, 95000.0)

In [12]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
1,S,GLOBAL,"4,500",18.60,"83,700.00",C18.40,SET50,2,2022-02-02
2,S,JASIF,"10,000",9.85,"98,500.00",C9.80,SET,2,2022-05-20


In [13]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(152000, 182200.0, 334200.0)

### After call ord-log (must call everytime that orders change)

In [14]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [15]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [16]:
mb = (df.trans == 'Buy') & (df.spd >= -3)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
6,B,IVL,-3,P50%3M,SET50,3000,46.25,47.0,-0.5,-1.05%,1,2022-05-31,Buy,138750.0
8,B,MCS,-2,52WL,SET,5000,11.40,11.6,-0.1,-0.85%,2,2022-03-15,Buy,57000.0


In [17]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [18]:
ms = (df.trans == 'Sell') & (df.spd <= 3)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
14,S,GLOBAL,3,C18.40,SET50,4500,18.60,18.30,-0.2,-1.08%,2,2022-02-02,Sell,83700.0
18,S,SIS,2,C28.50,SET,3600,28.75,28.25,1.5,+5.61%,1,2022-03-10,Sell,103500.0


In [19]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
6,B,IVL,-3,P50%3M,SET50,3000,46.25,47.00,-0.5,-1.05%,1,2022-05-31,Buy,138750.0
8,B,MCS,-2,52WL,SET,5000,11.40,11.60,-0.1,-0.85%,2,2022-03-15,Buy,57000.0
14,S,GLOBAL,3,C18.40,SET50,4500,18.60,18.30,-0.2,-1.08%,2,2022-02-02,Sell,83700.0
18,S,SIS,2,C28.50,SET,3600,28.75,28.25,1.5,+5.61%,1,2022-03-10,Sell,103500.0


In [20]:
df[mb | ms].shape[0]

4

In [21]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
7,KBANK,Buy,"217,500.00",-10
10,SNC,Buy,"148,500.00",-18
9,SINGER,Buy,"141,000.00",-5
6,IVL,Buy,"138,750.00",-3
3,BANPU,Buy,"138,000.00",-10


### Select source of orders between these two

In [22]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
6,Buy,IVL,3000,46.25,1,-3,47.00,-0.5,-1.05%,P50%3M,SET50,2022-05-31
8,Buy,MCS,5000,11.40,2,-2,11.60,-0.1,-0.85%,52WL,SET,2022-03-15
14,Sell,GLOBAL,4500,18.60,2,3,18.30,-0.2,-1.08%,C18.40,SET50,2022-02-02
18,Sell,SIS,3600,28.75,1,2,28.25,1.5,+5.61%,C28.50,SET,2022-03-10


In [23]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
8,Buy,MCS,5000,11.40,2,-2,11.6,-0.1,-0.85%,52WL,SET,2022-03-15
14,Sell,GLOBAL,4500,18.60,2,3,18.3,-0.2,-1.08%,C18.40,SET50,2022-02-02
15,Sell,JASIF,10000,9.85,2,5,9.6,0.0,0.00%,C9.80,SET,2022-05-20


In [24]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process